# Project Mobile Robot

(Prof. Francesco Mondada)

Students :  
Roxane Pangaud, Sélim Kamal, Yann Boudigou, Harsh Jaiswal

## Introduction

Our environment is an open space (white is preferred) on which we add black obstacle of random shapes. We founded more intresting to work with random obstacle than already define one (squares for exemple). 

We then use vision to transform the real map in a grid map. We chose to divide the space (150cm x 150cm) by 50x50 in squares, based on the pixels of an image taken by a smartphone. So in the end the space is divided in small squares of 3cm by 3cm.

We also use the vision to find the coordinates of the Thymio and the goal, as well as the angle of the robot.

After that, we use A* algorithm to find a path between the "start" (Thymio) and the "goal" (a target green and yellow).

Finally we use odometry and local avoidance in order for the Thymio to navigate in the environment. We sometimes check the position of the Thymio with the camera. And since we check in the camera sometimes, we can change the goal position if the robot isn't too close to finish it's run!

## Vision

*Yann ? Harsh?*

## Filtering

*Yann ? Harsh?*

## Global Navigation

*Sélim ?*

## Motion Contol

For the motion control, we got inspired by the odometry done in the 6th exercise session. We first calibrate the Thymio, to understand what time it needed for turning 360 degrees. We tried to turn the Thymio and stoped at 340°, we then measured the time. With those data, we used rule of tree and tried to turn the Thymio in different angle. After some time tunning, it was good enough for our application. 

After that, we did the same for the robot's advance by measuring how much it advances for a certain amount of time.

Here are the global variable used for odometry and more generally the motion:

In [1]:
## Global Variable for odometry
CALIBRATION_TIME_ANGLE = 9.6
CALIBRATION_ANGLE = 340
CALIBRATION_TIME = 3.47 # time needed to do the DISTANCE_CALLIBRATION
DISTANCE_CALIBRATION = 10.7

## Global variable for motion
COTE_CARRE_MAP = 150 / MAP_SIZE  # MAP_SIZE = 50, the nb of squares in the rows or colomns (it's a squared map) of the map

NameError: name 'MAP_SIZE' is not defined

With this done, we created a function called "robot_evaluation" that need the position and the angle of the robot, as well as the position of the curent target and it returns *gamma*, the angle that the robot must turn to be aligned with the target in degrees, *direction* ("left","right" or "forward"), which indicate wich direction to turn and *distance*, the distance between the Thymio and the target (in cm). To calculate all of this, we use trigonometry for the different cases of relative positions between the Thymio and the target.

After that, we have all what we need to make the robot move. We implemented then 2 functions in order to move the Thymio: "robot_turn" and "robot_move_forward". The first one takes *gamma*, *direction* and the angle of the robot (*alpha*) and moves it in the direction and the angle (*gamma*) desired, then it changes alpha, regarding of how much it thinks the Thymio has moved (*gamma*) and the direction to "forward" since we now are . The second one makes the Thymio move forward. The function takes *distance* and the curent position of the robot and the target, and then move the robot forward by *distance* centimeters. Then it updates the position of the Thymio by saying that it is now on the target (target position = robot position).

In [ ]:
def robot_turn(gamma, direction,alpha): #return alpha, direction, obstacle_seen (bool: 0->free)
    gamma = abs(gamma)
    time_turn = CALIBRATION_TIME_ANGLE*gamma/CALIBRATION_ANGLE
    obstacle_seen = 0
    
    if direction == "right":
        robot.set_var("motor.left.target", MOTOR_SPEED)
        robot.set_var("motor.right.target", int(int(2**16)-MOTOR_SPEED))
        t0 =time.time()
        while time.time()-t0 < time_turn :
            if (check_avoidance()) :
                obstacle_seen = 1
                avoid_obstacle()
                return alpha, direction, obstacle_seen
        
        alpha = angle_between_pi(alpha-math.radians(gamma))
        
    if direction == "left":
        robot.set_var("motor.left.target", int(int(2**16)-MOTOR_SPEED))
        robot.set_var("motor.right.target", MOTOR_SPEED)
        t0 =time.time()
        while time.time()-t0 < time_turn :
            if (check_avoidance()) :
                obstacle_seen = 1
                avoid_obstacle()
                return alpha, direction, obstacle_seen
        
        alpha = angle_between_pi(alpha+math.radians(gamma))
    
    direction = "forward"
    
    robot_stop()
    
    return alpha, direction, obstacle_seen

## Local navigation 

Local navigation is mainly about 